# Custom Chatbot Project

The chosen data is pulled from Calgary's open data service \
URL: https://data.calgary.ca/Services-and-Amenities/Calgary-Eats-Farm-Stand-Program-Locations/mhqb-ca5d \
This dataset contains the location of foodstands in Calgary and was recently updated with new data.\
While one could query the location of Calgary foodstands using the openai models, its knowledge on this particular topic would be outdated.

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
import os
import numpy as np
import pandas as pd
import openai
openai.api_key = "YOUR API KEY"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 1 #100

data_folder = "data"
data_file_path="Calgary_Eats_Farm_Stand_Program_20240610.csv"
embed_file_path="calgary_farmstand_embeddings.csv"
df_farm = pd.read_csv(os.path.join(data_folder,data_file_path))


In [2]:
df_farm.head(2)

,TYPE,LOCATION_NAME,ADDRESS,HOURS,DURATION,COMMUNITY,SECTOR,VENDOR_NAME,VENDOR_DESCRIPTION,PRODUCT_DESCRIPTION,CREATE_DT_UTC,MOD_DT_UTC,POINT,GLOBALID
0,Farm Stand,Bridgeland: Columbus Manor,40 11A Street NE,Monday 11:30am - 1:30pm,Year-round,Bridgeland/ Riverside,Northeast,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",2022/02/10,2024/04/11,POINT (-114.0344284 51.0504938),0108acff-ad8b-4625-8f9a-b02b684cef33
1,Farm Stand,Hillhurst Sunnyside: Community Association,1320 5 Ave NW,Monday 4pm-6pm,Year-round,Hillhurst,Northwest,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",2023/05/04,2024/04/11,POINT (-114.0925564 51.057507),c71d0982-47c4-4501-a2f5-087de6473075


We will compose the data in sentence form and check the length of the content.

In [3]:
lst_columns_to_keep = ['LOCATION_NAME','ADDRESS','HOURS','DURATION','COMMUNITY','SECTOR','VENDOR_NAME','VENDOR_DESCRIPTION','PRODUCT_DESCRIPTION']


df_farm = df_farm[df_farm['DURATION']!='remove'].copy()
df_farm['LOCATION_NAME_SUFFIX'] = df_farm['LOCATION_NAME'].str.split(':').str[-1]

df_farm['text'] = df_farm['VENDOR_NAME'] + " has a farmstand located in the " + df_farm['SECTOR'] \
+ " quadrant of Calgary at the " + df_farm['LOCATION_NAME_SUFFIX'] + " in the community of " + df_farm['COMMUNITY'] \
+ '. Their address is ' + df_farm['ADDRESS'] + ". They operate " + df_farm['DURATION'].str.replace('Open ','') \
+ ", " + df_farm['HOURS'] + ". They sell " + df_farm['PRODUCT_DESCRIPTION'] + ". According to them, " + df_farm['VENDOR_DESCRIPTION'] 

df_farm = df_farm[lst_columns_to_keep+['text']]

In [4]:
print(f"The max length is {df_farm['text'].str.len().max()}")
df_farm.loc[:,'text'].head(2).tolist()

The max length is 835


['Fresh Routes has a farmstand located in the Northeast quadrant of Calgary at the  Columbus Manor in the community of Bridgeland/ Riverside. Their address is 40 11A Street NE. They operate Year-round, Monday 11:30am - 1:30pm. They sell Fresh fruit, vegetables, whole grains, and eggs.. According to them, Fresh Routes is a not-for-profit social enterprise. Their Mobile Grocery Stores bring healthy, fresh, and affordable food into neighbourhoods facing barriers: allowing choice, maintaining dignity, and building community. Their mission is to ensure that everyone has access to affordable, nutritious food and is informed to make healthy decisions and live a healthy life!',
 'Fresh Routes has a farmstand located in the Northwest quadrant of Calgary at the  Community Association in the community of Hillhurst. Their address is 1320 5 Ave NW. They operate Year-round, Monday 4pm-6pm. They sell Fresh fruit, vegetables, whole grains, and eggs.. According to them, Fresh Routes is a not-for-profit

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [5]:

def create_or_use_embeddings(df):
    if os.path.exists(os.path.join(data_folder,embed_file_path)):
        
        df_emb = pd.read_csv(os.path.join(data_folder,embed_file_path), index_col=0)
        df_emb["embeddings"] = df_emb["embeddings"].apply(eval).apply(np.array)
        df_concat = pd.concat([df,df_emb['embeddings']],axis=1)
        return df_concat
    else:
        embeddings = []
        for i in range(0, len(df), batch_size):
            # Send text data to OpenAI model to get embeddings
            response = openai.Embedding.create(
                input=df_farm.iloc[i:i+batch_size]["text"].tolist(),
                engine=EMBEDDING_MODEL_NAME
            )

            # Add embeddings to list
            embeddings.extend([data["embedding"] for data in response["data"]])
        df_farm["embeddings"] = embeddings
        df_farm[["embeddings"]].to_csv(os.path.join(data_folder,embed_file_path))
        return df_farm

In [6]:
# Add embeddings list to dataframe

df_farm_embed = create_or_use_embeddings(df_farm)
df_farm_embed.head(2)

,LOCATION_NAME,ADDRESS,HOURS,DURATION,COMMUNITY,SECTOR,VENDOR_NAME,VENDOR_DESCRIPTION,PRODUCT_DESCRIPTION,text,embeddings
0,Bridgeland: Columbus Manor,40 11A Street NE,Monday 11:30am - 1:30pm,Year-round,Bridgeland/ Riverside,Northeast,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the No...,"[0.004099639132618904, -0.03901916369795799, -..."
1,Hillhurst Sunnyside: Community Association,1320 5 Ave NW,Monday 4pm-6pm,Year-round,Hillhurst,Northwest,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the No...,"[0.007366276811808348, -0.03800453245639801, -..."


In [7]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy


In [8]:

get_rows_sorted_by_relevance("What does Fresh Routes sell at the university", df_farm_embed).head(5)


,LOCATION_NAME,ADDRESS,HOURS,DURATION,COMMUNITY,SECTOR,VENDOR_NAME,VENDOR_DESCRIPTION,PRODUCT_DESCRIPTION,text,embeddings,distances
19,University of Calgary: The Nest,250 Collegiate Blvd NW,Thursday 11:30am - 1:30pm,Year-round,University of Calgary,Northwest,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the No...,"[0.010012490674853325, -0.05308948829770088, -...",0.123969
4,Alberta University of the Arts,1407 14 Ave NW,Friday 11:30am-1:30pm,Year-round,Hillhurst,Northwest,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the No...,"[0.006390706170350313, -0.04200666770339012, -...",0.127064
18,Foothills Hospital: Health Sciences Centre,3330 Hospital Dr NW,Wednesday 11am-1pm,Year-round,St. Andrews Heights,Northwest,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the No...,"[0.011498209089040756, -0.035065751522779465, ...",0.132434
28,Manchester: Manchester Housing Building,5757 2 St SW,Wednesday 6pm - 7:30pm,Year-round,Manchester,Southeast,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the So...,"[0.0065203458070755005, -0.03766700625419617, ...",0.134464
9,East Village: Murdoch Manor,808 5 St SE,Wednesday 2:30pm-4pm,Year-round,East Village,Southeast,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the So...,"[0.007067762315273285, -0.046364519745111465, ...",0.140130


In [9]:
get_rows_sorted_by_relevance("Which Calgary communities have farmstands?", df_farm_embed).head(5)

,LOCATION_NAME,ADDRESS,HOURS,DURATION,COMMUNITY,SECTOR,VENDOR_NAME,VENDOR_DESCRIPTION,PRODUCT_DESCRIPTION,text,embeddings,distances
17,Edgemont: Community Association,33 Edgevalley Cir NW,Thursday 3:30pm - 7pm,Open June 13,Edgemont,Northwest,FreshPal Farms,"FreshPal Farms is located near Olds, Alberta a...","Many varieties of tomatoes, cucumbers, lettuce...",FreshPal Farms has a farmstand located in the ...,"[0.010477082803845406, -0.026442160829901695, ...",0.142082
12,Woodlands: Woodcreek Community Association,1991 Woodview Drive SW,Friday 2:30pm-5:30pm,Open Mid-July to October (Check back for details),Woodlands,Southwest,Tetzlaff Family Farm,We are a small family run farm located near Hi...,"Peas, beans, squash, green onion, potatoes, on...",Tetzlaff Family Farm has a farmstand located i...,"[-0.007897358387708664, -0.031398970633745193,...",0.146243
15,Springbank Hill: Valleyview Community Church,7655 26 Ave SW,Saturday 4pm-6pm,Open June 15,Springbank Hill,Northwest,FreshPal Farms,"FreshPal Farms is located near Olds, Alberta a...","Many varieties of tomatoes, cucumbers, lettuce...",FreshPal Farms has a farmstand located in the ...,"[0.007765172980725765, -0.02934255637228489, -...",0.148092
7,Ranchlands: Community Association,7713 Ranchview Dr NW,Monday 2pm-6pm,May 20 to October,Ranchlands,Northwest,Shirley's Greenhouse,"We are a 36,000 square foot greenhouse located...","Many varieties of tomatoes, cucumbers, hot pep...",Shirley's Greenhouse has a farmstand located i...,"[-0.000984629150480032, -0.03637198358774185, ...",0.148264
1,Hillhurst Sunnyside: Community Association,1320 5 Ave NW,Monday 4pm-6pm,Year-round,Hillhurst,Northwest,Fresh Routes,Fresh Routes is a not-for-profit social enterp...,"Fresh fruit, vegetables, whole grains, and eggs.",Fresh Routes has a farmstand located in the No...,"[0.007366276811808348, -0.03800453245639801, -...",0.148680


In [10]:
get_rows_sorted_by_relevance("Which independent farmstands in Calgary sell vegetables at community associations?", df_farm_embed).head(5)

,LOCATION_NAME,ADDRESS,HOURS,DURATION,COMMUNITY,SECTOR,VENDOR_NAME,VENDOR_DESCRIPTION,PRODUCT_DESCRIPTION,text,embeddings,distances
21,Parkland: Community Association,505 Parkvalley Rd SE,Saturday 10am - 2pm,May 4 to October,Parkland,Southeast,Shirley's Greenhouse,"We are a 36,000 square foot greenhouse located...","Many varieties of tomatoes, cucumbers, hot pep...",Shirley's Greenhouse has a farmstand located i...,"[0.0015002023428678513, -0.03353001922369003, ...",0.133608
7,Ranchlands: Community Association,7713 Ranchview Dr NW,Monday 2pm-6pm,May 20 to October,Ranchlands,Northwest,Shirley's Greenhouse,"We are a 36,000 square foot greenhouse located...","Many varieties of tomatoes, cucumbers, hot pep...",Shirley's Greenhouse has a farmstand located i...,"[-0.000984629150480032, -0.03637198358774185, ...",0.135559
17,Edgemont: Community Association,33 Edgevalley Cir NW,Thursday 3:30pm - 7pm,Open June 13,Edgemont,Northwest,FreshPal Farms,"FreshPal Farms is located near Olds, Alberta a...","Many varieties of tomatoes, cucumbers, lettuce...",FreshPal Farms has a farmstand located in the ...,"[0.010477082803845406, -0.026442160829901695, ...",0.137387
29,Kingsland: Community Association,505 78 Avenue SW,Tuesday 10am - 1pm,Open April 30 to October,Kingsland,Southeast,Broxburn Vegetables,Broxburn Vegetables is located just outside of...,"Vine-ripened tomatoes, long English cucumbers,...",Broxburn Vegetables has a farmstand located in...,"[-0.0006160232587717474, -0.049268338829278946...",0.141006
10,Copperfield: Community Association,6 Copperstone Way SE,Tuesday 2pm-6pm,May 21 to October,Copperfield,Southeast,Shirley's Greenhouse,"We are a 36,000 square foot greenhouse located...","Many varieties of tomatoes, cucumbers, hot pep...",Shirley's Greenhouse has a farmstand located i...,"[-0.00026973659987561405, -0.03132142871618271...",0.141115


In [11]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
    Answer the question based on the context below, and if the question
    can't be answered based on the context, say "I don't know"

    Context: 

    {}

    ---

    Question: {}
    Answer:
    """
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)
    

In [12]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""
        

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [13]:
question1 = "Which Calgary communities have farmstands?"

question1_prompt = f"""
Question: "{question1}"
Answer:
"""
question1_answer = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=question1_prompt,
    max_tokens=350
)["choices"][0]["text"].strip()
print(question1_answer)

Unfortunately, there is no definitive list of Calgary communities with farmstands, as farmstands are often operated by individual farmers or small businesses and may not be officially registered or listed. However, here are some communities known to have regular farmstands or farmers markets:

1. Hillhurst-Sunnyside: This community has a weekly farmers market called Sunnyside Market, where local farmers and vendors sell fresh produce, crafts, and more.

2. Bridgeland-Riverside: The Bridgeland Riverside Farmers' Market is a popular weekly market with vendors selling fresh produce, baked goods, and handmade goods.

3. Inglewood: The Inglewood Night Market is a summer market that takes place on the first Friday of the month, featuring local farmers and vendors selling their goods.

4. Brentwood: The Brentwood Farmers' Market takes place every Wednesday from June to September, featuring local farmers and vendors selling fresh produce, and other items.

5. Mahogany: The Mahogany Market is a

In [14]:
custom_question1_answer = answer_question(question1, df_farm_embed[['text','embeddings']])
print(custom_question1_answer)

FreshPal Farms has a farmstand located in the Northwest quadrant of Calgary at the Community Association in the community of Edgemont.

Tetzlaff Family Farm has a farmstand located in the Southwest quadrant of Calgary at the Woodcreek Community Association in the community of Woodlands.

FreshPal Farms has a farmstand located in the Northwest quadrant of Calgary at the Valleyview Community Church in the community of Springbank Hill.

Shirley's Greenhouse has a farmstand located in the Northwest quadrant of Calgary at the Community Association in the community of Ranchlands.

Fresh Routes has a farmstand located in the Northwest quadrant of Calgary at the Community Association in the community of Hillhurst.

Shirley's Greenhouse has a farmstand located in


### Question 2

In [15]:
question2 = "Which independent farmstands in Calgary sell vegetables at community associations?"

question2_prompt = f"""
Question: "{question2}"
Answer:
"""
question2_answer = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=question2_prompt,
    max_tokens=350
)["choices"][0]["text"].strip()
print(question2_answer)

1. Big Mountain Farm
2. Springbank Heritage Farm
3. Pekisko Creek Farm
4. Little Creek Gardens
5. The Homestead Farms
6. The Cider Goose
7. Farmer's Fresh Market
8. Reclaim Urban Farm
9. The Prairie Farm Stand
10. Sundog Organic Farm
11. Cherry Pit Farm
12. Eagle Creek Farms
13. Tyner Ridge Organic Farm
14. Sage Garden Market
15. The Village Farmstand.


In [16]:
custom_question2_answer = answer_question(question2, df_farm_embed[['text','embeddings']])
print(custom_question2_answer)

1. Shirley's Greenhouse at the Community Association in the community of Parkland
    2. Broxburn Vegetables at the Community Association in the community of Kingsland
    3. Shirley's Greenhouse at the Community Association in the community of Copperfield
    4. Shirley's Greenhouse at the Prominence Park in the community of Patterson
    5. Broxburn Vegetables at the Community Association in the community of Montgomery
    6. FreshPal Farms at the Valleyview Community Church in the community of Springbank Hill
    7. Broxburn Vegetables at the community Association in the community of Wildwood
    8. Broxburn Vegetables at the Community Association in the community of Winston Heights.
